In [ ]:
%pip install imgaug

In [2]:
import os
import sys
import json
import imgaug
import datetime
import numpy as np
import skimage.draw

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' 
np.bool = np.bool_

# Root directory of the project
# ROOT_DIR = os.path.abspath("../")
ROOT_DIR = os.getcwd()
ROOT_DIR = os.path.normpath(ROOT_DIR)
print("Working Dir: ", ROOT_DIR)
DATASET_DIR = os.path.join(ROOT_DIR, 'Datasets/logoDetector/')
DATASET_DIR = os.path.normpath(DATASET_DIR)
print("Dataset Dir: ", DATASET_DIR)

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn2.config import Config
from mrcnn2 import model as modellib, utils

# Path to trained weights file
COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "logoDetectorModel/preTrainedModel/mask_rcnn_coco.h5")

# Directory to save logs and model checkpoints, if not provided
# through the command line argument --logs
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logoDetectorModel/trainingLogs")

############################################################
#  Configurations
############################################################


class TrainingConfig(Config):
    """Configuration for training on the toy  dataset.
    Derives from the base Config class and overrides some values.
    """
    # Give the configuration a recognizable name
    NAME = "sewez"

    # We use a GPU with 12GB memory, which can fit two images.
    # Adjust down if you use a smaller GPU.
    IMAGES_PER_GPU = 1

    # Number of classes (including background)
    NUM_CLASSES = 1 + 3  # Background + sewez_tshirt, sewez_tshirt_logo, sewez_tshirt_brand

    # Number of training steps per epoch
    STEPS_PER_EPOCH = 5

    # Skip detections with < 90% confidence
    DETECTION_MIN_CONFIDENCE = 0.9

    # Training Learning Rate
    LEARNING_RATE = 0.001


############################################################
#  Dataset
############################################################

class TrainingDataset(utils.Dataset):

    def load_training_dataset(self, dataset_dir, subset):
        """Load a subset of the Balloon dataset.
        dataset_dir: Root directory of the dataset.
        subset: Subset to load: train or val
        """
        # Add classes. We have only one class to add.
        self.add_class("sewez", 1, "sewez_tshirt")
        self.add_class("sewez", 2, "sewez_tshirt_logo")
        self.add_class("sewez", 3, "sewez_tshirt_brand")

        # Train or validation dataset?
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        # Load annotations
        # VGG Image Annotator (up to version 1.6) saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }
        # We mostly care about the x and y coordinates of each region
        # Note: In VIA 2.0, regions was changed from a dict to a list.
        annotations = json.load(open(os.path.join(dataset_dir, "annotation.json")))
        annotations = list(annotations.values())  # don't need the dict keys

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
        annotations = [a for a in annotations if a['regions']]

        # Add images
        for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. These are stores in the
            # shape_attributes (see json format above)
            # The if condition is needed to support VIA versions 1.x and 2.x.
            if type(a['regions']) is dict:
                polygons = [r['shape_attributes'] for r in a['regions'].values()]
            else:
                polygons = [r['shape_attributes'] for r in a['regions']] 

            # iterate names
            objects = [s['region_attributes']['names'] for s in a['regions']]
            name_dict = {"sewez_tshirt": 1, "sewez_tshirt_logo": 2, "sewez_tshirt_brand": 3}
            num_ids = [name_dict[a] for a in objects]

            # load_mask() needs the image size to convert polygons to masks.
            # Unfortunately, VIA doesn't include it in JSON, so we must read
            # the image. This is only managable since the dataset is tiny.
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "sewez",
                image_id=a['filename'],  # use file name as a unique image id
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids)

    def load_mask(self, image_id):
        """Generate instance masks for an image.
        Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        class_ids: a 1D array of class IDs of the instance masks.
        """
        # If not a Dog-Cat dataset image, delegate to parent class.
        image_info = self.image_info[image_id]
        if image_info["source"] != "sewez":
            return super(self.__class__, self).load_mask(image_id)

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
        info = self.image_info[image_id]
        if info["source"] != "sewez":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

            mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
        # Map class names to class IDs.
        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids #np.ones([mask.shape[-1]], dtype=np.int32)

    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


def train(model):
    """Train the model."""
    # Training dataset.
    dataset_train = TrainingDataset()
    dataset_train.load_training_dataset(DATASET_DIR, "train")
    dataset_train.prepare()

    # Validation dataset
    dataset_val = TrainingDataset()
    dataset_val.load_training_dataset(DATASET_DIR, "val")
    dataset_val.prepare()

    # *** This training schedule is an example. Update to your needs ***
    # Since we're using a very small dataset, and starting from
    # COCO trained weights, we don't need to train too long. Also,
    # no need to train all layers, just the heads should do it.
    print("Training network heads")
    # model.train(dataset_train, dataset_val,
    #             learning_rate=config.LEARNING_RATE,
    #             epochs=30,
    #             layers='heads')
    model.train(dataset_train, dataset_val,
                learning_rate=config.LEARNING_RATE,
                epochs=300,
                layers='heads', #layers='all', 
                augmentation = imgaug.augmenters.Sequential([ 
                imgaug.augmenters.Fliplr(1), 
                imgaug.augmenters.Flipud(1), 
                imgaug.augmenters.Affine(rotate=(-45, 45)), 
                imgaug.augmenters.Affine(rotate=(-90, 90)), 
                imgaug.augmenters.Affine(scale=(0.5, 1.5)),
                imgaug.augmenters.Crop(px=(0, 10)),
                imgaug.augmenters.Grayscale(alpha=(0.0, 1.0)),
                imgaug.augmenters.AddToHueAndSaturation((-20, 20)), # change hue and saturation
                imgaug.augmenters.Add((-10, 10), per_channel=0.5), # change brightness of images (by -10 to 10 of original value)
                imgaug.augmenters.Invert(0.05, per_channel=True), # invert color channels
                imgaug.augmenters.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)), # sharpen images
                ]
                ))

############################################################
#  Training
############################################################

config = TrainingConfig()
model = modellib.MaskRCNN(mode="training", config=config,
                                  model_dir=DEFAULT_LOGS_DIR)

weights_path = COCO_WEIGHTS_PATH
        # Download weights file
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

train(model)

Working Dir:  /content
Dataset Dir:  /content/Datasets/logoDetector
Training network heads

Starting at epoch 0. LR=0.001

Checkpoint Path: /content/logoDetectorModel/trainingLogs/sewez20240618T0554/mask_rcnn_sewez_{epoch:04d}.h5
Selecting layers to train
fpn_c5p5               (Conv2D)
fpn_c4p4               (Conv2D)
fpn_c3p3               (Conv2D)
fpn_c2p2               (Conv2D)
fpn_p5                 (Conv2D)
fpn_p2                 (Conv2D)
fpn_p3                 (Conv2D)
fpn_p4                 (Conv2D)
rpn_model              (Functional)
mrcnn_mask_conv1       (TimeDistributed)
mrcnn_mask_bn1         (TimeDistributed)
mrcnn_mask_conv2       (TimeDistributed)
mrcnn_mask_bn2         (TimeDistributed)
mrcnn_class_conv1      (TimeDistributed)
mrcnn_class_bn1        (TimeDistributed)
mrcnn_mask_conv3       (TimeDistributed)
mrcnn_mask_bn3         (TimeDistributed)
mrcnn_class_conv2      (TimeDistributed)
mrcnn_class_bn2        (TimeDistributed)
mrcnn_mask_conv4       (TimeDistributed)
m

/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/legacy/gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/300
5/5 [==============================] - ETA: 0s - batch: 2.0000 - size: 1.0000 - loss: 5.5892 - rpn_class_loss: 0.2443 - rpn_bbox_loss: 1.9748 - mrcnn_class_loss: 1.0375 - mrcnn_bbox_loss: 1.2323 - mrcnn_mask_loss: 1.1002    

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training_v1.py:2335: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


5/5 [==============================] - 115s 25s/step - batch: 2.0000 - size: 1.0000 - loss: 5.5892 - rpn_class_loss: 0.2443 - rpn_bbox_loss: 1.9748 - mrcnn_class_loss: 1.0375 - mrcnn_bbox_loss: 1.2323 - mrcnn_mask_loss: 1.1002 - val_loss: 2.5968 - val_rpn_class_loss: 0.1176 - val_rpn_bbox_loss: 0.6231 - val_mrcnn_class_loss: 0.3464 - val_mrcnn_bbox_loss: 1.0258 - val_mrcnn_mask_loss: 0.4839
Epoch 2/300
5/5 [==============================] - 86s 18s/step - batch: 2.0000 - size: 1.0000 - loss: 2.7322 - rpn_class_loss: 0.0818 - rpn_bbox_loss: 0.9693 - mrcnn_class_loss: 0.2661 - mrcnn_bbox_loss: 0.9339 - mrcnn_mask_loss: 0.4812 - val_loss: 2.3351 - val_rpn_class_loss: 0.0641 - val_rpn_bbox_loss: 0.5031 - val_mrcnn_class_loss: 0.3972 - val_mrcnn_bbox_loss: 0.9525 - val_mrcnn_mask_loss: 0.4183
Epoch 3/300
2/5 [===========>..................] - ETA: 37s - batch: 0.5000 - size: 1.0000 - loss: 2.5183 - rpn_class_loss: 0.1175 - rpn_bbox_loss: 0.7045 - mrcnn_class_loss: 0.2831 - mrcnn_bbox_loss: 

IndexError: index 1565 is out of bounds for axis 1 with size 1565